In [12]:
import numpy as np

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
sess.run(tf.global_variables_initializer())

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9197


In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [9]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.12
step 1, training accuracy 0.12
step 2, training accuracy 0.12
step 3, training accuracy 0.12
step 4, training accuracy 0.12
step 5, training accuracy 0.12
step 6, training accuracy 0.12
step 7, training accuracy 0.12
step 8, training accuracy 0.12
step 9, training accuracy 0.12
step 10, training accuracy 0.12
step 11, training accuracy 0.12
step 12, training accuracy 0.12
step 13, training accuracy 0.12
step 14, training accuracy 0.12
step 15, training accuracy 0.12
step 16, training accuracy 0.12
step 17, training accuracy 0.12
step 18, training accuracy 0.12
step 19, training accuracy 0.12
step 20, training accuracy 0.12
step 21, training accuracy 0.12
step 22, training accuracy 0.12
step 23, training accuracy 0.12
step 24, training accuracy 0.12
step 25, training accuracy 0.12
step 26, training accuracy 0.12
step 27, training accuracy 0.12
step 28, training accuracy 0.12
step 29, training accuracy 0.12
step 30, training accuracy 0.12
step 31, training 

KeyboardInterrupt: 

In [10]:
batch = mnist.train.next_batch(1000)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1] , keep_prob: 1.0}))

0.993


In [13]:
hCoun = np.array([])
hHip = np.array([])
prediction=tf.argmax(y_conv,1)
pred_arr = np.array(prediction.eval(feed_dict={x: batch[0], keep_prob: 1.0}, session=sess))
print ("predictions", pred_arr)

predictions [7 1 3 7 1 2 4 1 3 5 1 9 0 8 4 4 2 7 0 1 5 9 5 5 3 6 4 5 8 4 4 3 7 3 1 4 2
 7 2 3 3 7 2 1 7 7 1 8 8 9 4 0 0 2 0 8 3 8 6 8 4 4 9 7 1 9 4 4 3 2 0 5 3 5
 8 2 3 8 1 0 9 0 7 1 2 5 7 0 1 1 6 5 7 2 7 6 7 0 8 9 3 4 7 0 4 0 3 9 4 4 6
 1 1 3 3 3 3 5 6 2 9 0 5 3 2 7 1 4 5 2 6 3 6 1 6 9 1 9 4 3 7 2 0 5 4 6 8 2
 1 8 9 4 7 8 6 1 1 9 2 0 8 7 4 9 2 6 6 9 3 8 7 3 3 4 5 8 8 4 0 5 0 6 9 9 2
 5 4 7 1 0 4 3 2 8 8 1 2 0 3 4 0 0 5 0 7 0 9 9 6 7 6 1 5 1 6 8 6 4 6 8 0 3
 0 2 2 9 0 6 2 9 3 6 1 2 0 3 2 3 7 9 7 1 4 7 8 0 8 7 0 2 7 8 7 8 6 7 8 0 8
 1 9 9 8 7 8 0 1 1 1 0 4 7 7 3 2 2 1 2 8 9 8 6 8 6 4 2 9 4 0 8 6 6 1 9 4 7
 2 1 6 1 6 0 7 9 3 4 9 2 1 5 6 5 2 3 3 2 3 6 0 9 3 5 5 5 0 9 1 4 2 8 7 3 8
 1 6 0 4 2 7 1 9 9 9 9 1 8 9 4 6 4 3 6 2 4 7 1 2 0 7 9 1 5 3 0 3 6 6 9 7 0
 8 2 5 7 1 7 6 8 1 7 7 1 5 3 7 3 7 4 0 6 3 8 7 6 2 8 5 7 3 5 0 3 0 9 0 2 1
 1 8 6 1 0 1 2 2 1 3 2 0 4 6 7 4 3 5 7 9 7 0 6 8 3 7 6 0 3 3 3 7 0 6 7 1 7
 7 7 1 0 5 9 8 8 1 6 1 3 1 6 3 5 5 1 1 1 5 2 1 2 7 2 0 7 1 7 9 1 9 3 1 6 6
 5 0 8 7 3 9 

In [14]:
probabilities=y_conv
print ("probabilities", probabilities.eval(feed_dict={x: batch[0], keep_prob: 1.0}, session=sess))

probabilities [[ -4.14085102  -1.05011356   0.08349202 ...,   6.7861495    0.25801906
   -3.11390138]
 [ -1.89093852  14.953722    -1.75297534 ...,  -0.17784409   0.36471677
   -2.37491035]
 [ -5.06178904   0.49821335  -0.20346951 ...,  -2.14653134   0.12587237
   -0.75983751]
 ..., 
 [ 13.42436504  -2.03389645   0.01791715 ...,   1.54920709  -3.59952188
   -3.81357622]
 [ -0.57244754   0.3060008   11.17753696 ...,   1.66045809   1.63934982
   -2.42117715]
 [ -0.59626245  -2.20399451  -2.15779948 ...,  -5.99761248  15.53746128
   -2.1171062 ]]
